In [3]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores.pinecone import Pinecone
import pinecone
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

In [4]:
#os.environ['ANTHROPIC_API_KEY'] = getpass("Enter Anthropic key:")
#os.environ['OPENAI_API_KEY'] = getpass("Enter OpenAI API key:")
#os.environ['PINECONE_API_KEY'] = getpass("Enter Pinecone API key:")
#os.environ['PINECONE_ENV'] = getpass("Enter Pinecone Environment:")

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') 
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENV = os.getenv('PINECONE_API_ENV')
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [5]:
# initialize embedding
model = 'text-embedding-ada-002'

# The OpenAIEmbeddings class is instantiated with two parameters: 
# 'model' and 'openai_api_key'. 'model' is the name of the model to be used 
# and 'openai_api_key' is the key for accessing the OpenAI API.
embed = OpenAIEmbeddings(
    model=model,
    openai_api_key=OPENAI_API_KEY
)

In [6]:
# Defines the field name in the data which contains the text to be embedded.
text_field = "symptoms"

# Initializes the Pinecone service with the provided API key and environment.
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

# Defines the name of the Pinecone index to be used.
index_name = "medical-qa-search"
index = pinecone.Index(index_name)

# Creates an instance of the Pinecone class. It uses the previously created index,
# the embed_query method of the previously created embed object, and the text field.
vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

In [7]:
prompt_template = """Use the following pieces of context to answer the question enclosed within  3 backticks at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Please provide an answer which is factually correct and based on the information retrieved from the vector store.
Please also mention any quotes supporting the answer if any present in the context supplied within two double quotes "" .

{context}

QUESTION:```{question}```
ANSWER:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context","question"])
#
chain_type_kwargs = {"prompt": PROMPT}

In [8]:
# Creates an instance of the ChatOpenAI class.
rag_llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-4',
    temperature=0.0
)

In [9]:
qa_chain = RetrievalQA.from_chain_type(llm=rag_llm,
                                       chain_type="stuff",
                                       chain_type_kwargs={"prompt": PROMPT},
                                       retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
                                       return_source_documents=True
                                      )

In [10]:
user_question = input("Please provide the symptoms here :")
result = qa_chain(user_question)

Please provide the symptoms here : What are the symptoms of anaphylaxis


In [11]:
result['result']

'The symptoms of anaphylaxis include skin reactions such as hives and itching, flushed or pale skin, low blood pressure (hypotension), constriction of airways and a swollen tongue or throat which can cause wheezing and trouble breathing, a weak and rapid pulse, nausea, vomiting, or diarrhea, and dizziness or fainting. Other symptoms can include severe dizziness, severe trouble breathing or loss of consciousness, tightening of the throat, shortness of breath or wheezing, runny nose, constriction of airways, swelling of the throat that makes it difficult to breathe, a severe drop in blood pressure (shock), rapid pulse, dizziness, lightheadedness, or loss of consciousness.'

In [12]:
result['source_documents'][0].page_content

'Anaphylaxis symptoms usually occur within minutes of exposure to an allergen. Sometimes however it can occur a half-hour or longer after exposure. Signs and symptoms include: Seek emergency medical help if you your child or someone else youre with has a severe allergic reaction. Dont wait to see if the symptoms go away. If the person having the attack carries an epinephrine autoinjector administer it right away. Even if symptoms improve after the injection you still need to go to an emergency room to make sure symptoms dont recur even without more exposure to your allergen. This second reaction is called biphasic anaphylaxis. Make an appointment to see your doctor if you or your child has had a severe allergy attack or signs and symptoms of anaphylaxis in the past. The diagnosis and long-term management of anaphylaxis are complicated so youll probably need to see a doctor who specializes in allergies and immunology. Skin reactions including hives and itching and flushed or pale skin L